In [3]:
## wam-up : numpy

import numpy as np
import math


x = np.linspace(-math.pi,math.pi,2000)
y = np.sin(x)


a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate=1e-6

for t in range(2000):
    y_pred = a + b * x + c* x**2 + d * x **3
    
    loss = np.square(y_pred -y).sum()
    if t % 1000 == 99:
        print(t,loss)
    
    grad_y_pred = 2.0 * (y_pred -y)
    
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()
    
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
    
print(f'Result : y = {a} +{b}x + {c}x^2 + {d}x^3')

    
    

99 1523.3393860714248
1099 43.06757716074328
Result : y = 0.03188831451340203 +0.838950623852515x + -0.005501264877405754x^2 + -0.09079988527496523x^3


In [10]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")

x = torch.linspace(-math.pi,math.pi,2000, device = device,dtype=dtype)
y = torch.sin(x)

a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learngin_rate = 1e-6
for t in range(2000):
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    
    loss = (y_pred - y).pow(2).sum()
    
    if t % 100 == 99:
        print(t,loss.item())
    
    loss.backward()
    
    with torch.no_grad():
        a -= learngin_rate * a.grad
        b -= learngin_rate * b.grad
        c -= learngin_rate * c.grad
        d -= learngin_rate * d.grad
        
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
        
print(f'Result : y = {a.item()} +{b.item()}x + {c.item()}x^2 + {d.item()}x^3')




99 1603.780029296875
199 1120.01123046875
299 783.5952758789062
399 549.4459228515625
499 386.3392639160156
599 272.62884521484375
699 193.2938232421875
799 137.90090942382812
899 99.1971664428711
999 72.13575744628906
1099 53.20216369628906
1199 39.94694900512695
1299 30.661449432373047
1399 24.153162002563477
1499 19.588932037353516
1599 16.38643455505371
1699 14.13825511932373
1799 12.559293746948242
1899 11.449851036071777
1999 10.669976234436035
Result : y = -0.04373520240187645 +0.8450115323066711x + 0.007545048836618662x^2 + -0.09166199713945389x^3


In [19]:
## define new autograd funtions
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx,input):
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 ** input)
    
    @staticmethod
    def backward(ctx,grad_output):
        input , = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input **2 -1)
    

dtype = torch.float
device = torch.device("cpu")

x = torch.linspace(-math.pi, math.pi, 2000, device=device)

a = torch.full((),0.0,device=device,dtype=dtype,requires_grad=True)
b = torch.full((),-1.0,device=device,dtype=dtype,requires_grad=True)
c = torch.full((),0.0,device=device,dtype=dtype,requires_grad=True)
d = torch.full((),0.3,device=device,dtype=dtype,requires_grad=True)


learning_rate=5e-6

for t in range(2000):
    P3 = LegendrePolynomial3.apply
    y_pred = a + b * x + P3(c + d * x)
    
    loss = (y_pred - y).pow(2).sum()
    
    if  t % 100 == 99:
        print(5,loss.item())
    
    loss.backward()
    with torch.no_grad():
        a -= learngin_rate * a.grad
        b -= learngin_rate * b.grad
        c -= learngin_rate * c.grad
        d -= learngin_rate * d.grad

        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
print(f'Result : y = {a.item()} +{b.item()}x + {c.item()}x^2 + {d.item()}x^3')


5 2786.662109375
5 1233.6942138671875
5 549.270263671875
5 433.5107727050781
5 408.83343505859375
5 401.8392333984375
5 398.5426940917969
5 396.7341613769531
5 395.65118408203125
5 394.95465087890625
5 394.4811096191406
5 394.1449890136719
5 393.8981018066406
5 393.71160888671875
5 393.5674743652344
5 393.4538879394531
5 393.36273193359375
5 393.28863525390625
5 393.2276611328125
5 393.17681884765625
Result : y = 0.2005510926246643 +0.24137042462825775x + 0.7049540877342224x^2 + 0.0237678624689579x^3


In [47]:
##nn module

import torch
import math

x = torch.linspace(-math.pi,math.pi,2000)
y = torch.sin(x)

p = torch.tensor([1,2,3])
print(x.shape)
xx = x.unsqueeze(-1).pow(p)


model = torch.nn.Sequential(
        torch.nn.Linear(3,1),
        torch.nn.Flatten(0,1)
    )


## Mean squared error 
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate=1e-3
optimizer=torch.optim.RMSprop(model.parameters(),lr=learning_rate)

for t in range(2000):
    y_pred = model(xx)
    
    loss = loss_fn(y_pred,y)
    
    if t % 100 == 99:
        print(t,loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

linear_layer = model[0]

print(f'Resut: y = {linear_layer.bias.item()} + {linear_layer.weight[:,0].item()}x + {linear_layer.weight[:,1].item()}x^2 + {linear_layer.weight[:,2].item()}x^3')







torch.Size([2000])
99 3909.035400390625
199 673.7952880859375
299 113.43264770507812
399 71.30746459960938
499 62.864501953125
599 51.9769172668457
699 38.986480712890625
799 26.127914428710938
899 16.257158279418945
999 10.871615409851074
1099 9.153132438659668
1199 8.835882186889648
1299 8.846351623535156
1399 8.842153549194336
1499 9.085731506347656
1599 9.027626991271973
1699 9.006871223449707
1799 8.90821647644043
1899 8.895101547241211
1999 8.920973777770996
Resut: y = 0.0004930508439429104 + 0.8562325835227966x + 0.0004930515424348414x^2 + -0.09383887052536011x^3


In [ ]:
## custom nn Modules


In [49]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        
    def forward(self,x):
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
    
    def string(self):
        return f'{self.a.item()} {self.b.item()} {self.c.item()} {self.d.item()} '

    


x = torch.linspace(-math.pi,math.pi,2000)
y = torch.sin(x)



model = Polynomial3()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)


for t in range(2000):
    y_pred = model(x)
    
    loss = criterion(y_pred,y)
    
    if t % 100 == 99:
        print(t,loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()

print(f'Result: {model.string()}')




99 2029.6297607421875
199 1402.4046630859375
299 970.8934936523438
399 673.6972045898438
499 468.7836608886719
599 327.3459167480469
699 229.61740112304688
799 162.02044677734375
899 115.21717071533203
999 82.77919006347656
1099 60.2757568359375
1199 44.64950180053711
1299 33.78886795043945
1399 26.233795166015625
1499 20.97368049621582
1599 17.30840492248535
1699 14.75238037109375
1799 12.968560218811035
1899 11.722719192504883
1999 10.851997375488281
Result: -0.04404335469007492 0.8398067355155945 0.007598210126161575 -0.09092166274785995 


In [43]:
l = torch.nn.Linear(3,1)
xx[:3]
ll = l.forward(xx[:3])
print(ll.shape)


f = torch.nn.Flatten(0,1)

f.forward(ll)


torch.Size([3, 1])


tensor([10.0439, 10.0121,  9.9803], grad_fn=<ViewBackward>)